In [1]:
import pandas
from SmartCredit.ContratoConcatenating import ContratoConcatenating
from SmartCredit.CreateWorkingSpreadSheet import CreateWorkingSpreadSheet
from core.util import renameColumn

In [ ]:
# PREPARACAO DA TABELA NOVA
file = 'BaseClientes-FEV2019.csv'
data = pandas.read_csv(file, delimiter=';',encoding='latin-1',dtype=str)
data = data.drop(1505656)
cancelIndex = data[data.loc[:,'con_sta'].values == 'C'].index
data = data.drop(cancelIndex)
data = data.reset_index(drop=True)
data.to_csv('BaseClientes_FEV19_cleaned.csv',sep=';')
#'L': 1110013, 'P': 17165, 'C': 9064, 'E': 344058, 'A': 25356, nan: 1
#sumAberto = 446929087.5199511
#sumAtraso = 63490187.31998122
#sumPago = 1726812902.3391006

## Definicao das variaveis relevantes

In [2]:
#filePathTorre = 'C:\\Users\\frederico\\Documents\\!!Prime-Systems\\Credit-score\\dados\\'
#filePathNote = 'C:\\Users\\admprime\\Documents\\PRIME\\SmartCredit\\'
#fileCrivo = filePathNote + 'DadosCrivo-NOV2018.csv'
#filePos = filePathNote + 'BaseClientes-FEV2019.csv'
#fileCrivo = filePathNote + 'DadosCrivo-Reduzido.csv'
#filePos = filePathNote + 'BaseClientes-Reduzida.csv'
fileCrivo = 'DadosCrivo-NOV2018.csv'
filePos = 'BaseClientes_FEV19_cleaned.csv'
contCon = ContratoConcatenating()
contCon.PRE_REFERENCE_COLUMN = 'Codigo proposta'
contCon.POS_REFERENCE_COLUMN = 'Con_Contrato'

## Remocao de individuos repetidos
 - Percorre a tabela inteira e agrupa os individuos repetidos.
 - Escolhe 1 dos individuos para ser mantido atraves de criterios especificos (consultar codigo para detalhes).
 - Salva o indice de todos os individuos exceto pela duplicata escolhida para ser mantida.
 - Os arquivos gerados sao os seguintes: 
     * self.PRE_REMOVE_FILE = 'removeCrivo.txt'
     * self.POS_REMOVE_FILE = 'removePosCredit.txt'
 - Esses dois arquivos serao utilizados para concatenar as tabelas nos passos seguintes.

In [3]:
[preData,posData] = contCon.getData(fileCrivo, filePos)
contCon.removeCrivoDuplicatesList(preData)
contCon.removePosCreditDuplicatesList(posData)
print('done')

done


# OPCIONAL - Checagem da remocao de linhas repetidas
- Ele aplica os arquivos 'removeCrivo.txt' e 'removePosCredit.txt' às tabelas.
- Em seguida, verifica se há alguma repeticao.

In [ ]:
#Opcional: checagem da remocao de linhas repetidas - ele aplica a remocao com os arquivos
print("CRIVO")
contCon.checkIfLinesWereCorrectlyRemoved(preData, contCon.PRE_REMOVE_FILE, contCon.PRE_REFERENCE_COLUMN)
print("\nBASE DE CLIENTES")
contCon.checkIfLinesWereCorrectlyRemoved(posData, contCon.POS_REMOVE_FILE, contCon.POS_REFERENCE_COLUMN)


# OPCIONAL - Impressao de exemplos de linhas onde houve repeticao
- Ver codigo para detalhes

In [ ]:
contCon.printDuplicates(preData, contCon.PRE_REFERENCE_COLUMN)
contCon.printDuplicates(posData, contCon.POS_REFERENCE_COLUMN)

# Concatenação da tabela
- Saida: 'FullConcatTable.csv'

In [4]:
#contCon.readAndConcatData(fileCrivo, filePos)

In [ ]:
#CONCATENACAO VERSAO 2
preData = contCon.createCleanDatabase(fileCrivo, contCon.PRE_REMOVE_FILE, contCon.PRE_REFERENCE_COLUMN)
preData.loc[:,'Codigo proposta'] = preData.loc[:,'Codigo proposta'].astype(int)
posData = contCon.createCleanDatabase(filePos, contCon.POS_REMOVE_FILE,contCon.POS_REFERENCE_COLUMN)
renameColumn(posData, 'Pedido_Jretail', 'Codigo proposta')
posData.loc[:,'Codigo proposta'] = posData.loc[:,'Codigo proposta'].astype(int)

In [ ]:
dataMerged = pandas.merge(preData, posData, on=['Codigo proposta'])
dataMerged.to_csv('FullConcatTable.csv',sep=';')

# Definição do critério de inadimplência
 - Inicia adicionando a coluna inadimplente ao conjunto de dados criaArquivoQueDefineInadimplencia(data)
 - Saida: CreditDefaultSemear.csv

In [ ]:
#dataFile = 'FullConcatTable.csv'
#data = pandas.read_csv(dataFile, delimiter=';',encoding='latin-1',dtype=str)
#createWork = CreateWorkingSpreadSheet()
#createWork.criaArquivoQueDefineInadimplencia(data)

In [76]:
# CREDIT DEFAULT VERSAO 2
dataMerged.loc[:,'DiasEmAtraso'] = dataMerged.loc[:,'DiasEmAtraso'].astype(int)
inadimplenteCol = []
for i in dataMerged.index:
    if dataMerged.loc[i,'DiasEmAtraso'] > 60:
        inadimplenteCol.append('1')
    else:
        inadimplenteCol.append('0')
dataMerged['Inadimplente'] = inadimplenteCol
dataMerged.to_csv('CreditDefaultSemear-v2.csv',sep=';')